In [11]:
from nodes.log import LoggerMixIn, LogMixIn

LoggerMixIn.setlevel(LogMixIn.INFO)

Create a simple CDN setup consisting of two clients, two caches and a single origin.

```
[client1] -┐┌-> [cache1] -┐
           ├┤             ├-> [cache3] --> [origin]
[client2] -┘└-> [cache2] -┘
```

The clients are requesting different profiles of Zipf distribution according to different arrival processes, the
caches are LFU Caches, meaning, they evict the least frequently used objects, and the origin is a simple origin.


In [12]:
from cdnsim.arrival import Arrival


class Poisson(Arrival):
    def __init__(self, lam: float, **kwargs):
        super().__init__(**kwargs)
        if lam <= 1:
            raise ValueError(f"lambda for poisson should be greater or equal to 1, got: {lam}")
        self.__lam = lam
        self.__rng = np.random.default_rng()

    def __next__(self) -> int:
        if self._tick <= 0:
            raise StopIteration
        self._tick -= 1
        return int(self.__rng.poisson(self.__lam, 1)[0])


In [13]:
from scipy.stats import zipfian, binom

from typing import Iterator, List, Tuple

import numpy as np

from cdnsim import Client
from cdnsim.arrival import Arrival
from cdnsim.requests import Requests


class ZipfClient(Client):

    def __init__(self, cbase: int, n: int, p: float, a: float, arrival: Arrival):
        self._cbase = cbase
        self._csize = binom.rvs(n, p, size=cbase)
        self._a = a
        self._arrival = arrival
        super().__init__()

    def _generate(self) -> Iterator[List[Tuple[str, Requests]]]:
        for k in self._arrival:
            r = np.unique(zipfian.rvs(self._a, self._cbase, size=k), return_counts=True)
            yield [(remote, request) for remote, request in zip(self.remotes,
                                                                Requests(freqs=list(r[1]),
                                                                         index={
                                                                             'time': np.full(self._cbase, self._tick),
                                                                             'size': self._csize,
                                                                             'content': list(r[0])}) // len(
                                                                    self.remotes))]


In [14]:
client1 = ZipfClient(cbase=20, n=50, p=0.5, a=1.6, arrival=Poisson(lam=50, ticks=10))
client2 = ZipfClient(cbase=100, n=200, p=0.3, a=1.1, arrival=Poisson(lam=80, ticks=10))

In [15]:
from cdnsim.cache.cache import Cache

from typing import cast


class PLFUCache(Cache):
    def __init__(self, size: int, **kwargs):
        super().__init__(**kwargs)
        self._size = size

    def _work(self) -> None:
        while recv := self._receive():
            requests = cast(Requests, sum(recv))  # <-- TODO: this can be abstracted in a higher layer
            misses = requests.sort_values(ascending=False).reset_index('size').prod(axis=1).cumsum().index

            for remote, request in zip(self.remotes, request[misses] // len(self.remotes)):
                self._send(remote, request)
        self._terminate()


In [16]:
cache1 = PLFUCache(size=10)
cache2 = PLFUCache(size=10)
cache3 = PLFUCache(size=20)


In [7]:
from typing import cast

from cdnsim.requests import BaseRequests
from nodes.log import LoggerMixIn
from nodes.node import LNode


class Origin(LoggerMixIn, LNode):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._requests = BaseRequests([], {'tick': [], 'content': []})

    def _work(self) -> None:
        while recv := self._receive():
            assert isinstance(recv, list) and len(recv) > 0, recv
            self._log(f"received {cast(BaseRequests, sum(recv)).sum()} requests")


In [8]:
origin = Origin()

In [9]:
# second, establish connections
client1.connect_to(cache1)
client1.connect_to(cache2)
client2.connect_to(cache1)
client2.connect_to(cache2)
cache1.connect_to(cache3)
cache2.connect_to(cache3)
cache3.connect_to(origin)

In [10]:
# third, run simulation
client1.start_all()
client1.join_all()


INFO:ZipfClient-1:Start ZipfClient-1
INFO:ZipfClient-2:Start ZipfClient-2
INFO:PLFUCache-3:Start PLFUCache-3
INFO:PLFUCache-4:Start PLFUCache-4
INFO:PLFUCache-5:Start PLFUCache-5
INFO:Origin-6:Start Origin-6
INFO:ZipfClient-1:Join ZipfClient-1
INFO:ZipfClient-2:Join ZipfClient-2
INFO:PLFUCache-3:Join PLFUCache-3
INFO:PLFUCache-4:Join PLFUCache-4
INFO:PLFUCache-5:Join PLFUCache-5
INFO:Origin-6:Join Origin-6
